### 1. Устанавливаем необходимые библиотеки

In [ ]:
%pip install -r requirements.txt

После установки библиотек необходимо перегазгрузить Kernel. Для этого в верхнем меню выберите Kernel -> Restart Kernel

### 2. Указываем необходимые параметры для работы с облаком

In [1]:
TOKEN = "" # IAM токен, подробнее - https://yandex.cloud/ru/docs/iam/operations/iam-token/create-for-sa
FOLDER_ID = "" # ваш folder_id в облаке

##### скачиваем сертификат

In [3]:
# здесь используется сертификат Yandex Cloud, но если вы также используете SSL, то установите свой
!mkdir -p .clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document .clickhouse/root.crt && \
chmod 0600 .clickhouse/root.crt

--2024-05-12 17:41:29--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 2a02:6b8::1d9, 213.180.193.243
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|2a02:6b8::1d9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3,5K) [application/x-x509-ca-cert]
Saving to: ‘.clickhouse/root.crt’

.clickhouse/root.cr 100%[===================>]   3,50K  --.-KB/s    in 0s      

2024-05-12 17:41:29 (69,9 KB/s) - ‘.clickhouse/root.crt’ saved [3579/3579]



### 3. Импортируем необходимые библиотеки и указываем данные подключений к сопутствующим сервисам

In [5]:
import requests
import json

FAST_API_HOST = 'http://127.0.0.1:8000'
CH_HOST = ''
CH_PORT = '8443'
CH_USER = 'admin'
CH_PASSWORD = ''
CA_CERT = 'root.crt' # путь к скачанному сертификату
FOLDER_ID = ''
S3_PREFIX = ''
S3_BUCKET = ''
S3_URL = 'https://storage.yandexcloud.net' # или какой-либо другой
S3_ACCESS_KEY_ID = ''
S3_SECRET_ACCESS_KEY = ''

### 4. Выполняем запрос к Clickhouse для проверки подключения

In [7]:
url = f"{FAST_API_HOST}/clickhouse_connection?host={CH_HOST}&port={PORT}&user={CH_USER}&password={CH_PASSWORD}&ca_cert={CA_CERT}"

payload = {}
headers = {
    'Cache-Control': 'no-cache',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip,deflate',
    'Connection': 'keep-alive'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text) # в случае успеха показывается текущая версия Clickhouse

"24.3.2.23"


### 5. Создаем индекс в Clickhouse для поиска релевантных документов 

In [9]:
url = f"{FAST_API_HOST}/embeddings?ch_host={CH_HOST}&ch_port={CH_PORT}&ch_user={CH_USER}&ch_password={CH_PASSWORD}&ch_ca_cert={CA_CERT}&folder_id={FOLDER_ID}&s3_bucket={S3_BUCKET}&s3_prefix={S3_PREFIX}&s3_url={S3_URL}/&token={TOKEN}&s3_access_key_id={S3_ACCESS_KEY_ID}&s3_secret_access_key={S3_SECRET_ACCESS_KEY}"

payload = {}
headers = {
    'Cache-Control': 'no-cache',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip,deflate',
    'Connection': 'keep-alive'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

null


### 6. Тестируем поиск по векторам для запроса 

###### Могу ли я использовать эти API в pytorch 2.3.0?
###### torch.jit.quantized APIs

###### torch.jit.quantized.quantize_rnn_cell_modules

###### torch.jit.quantized.quantize_rnn_modules
###### torch.jit.quantized.quantize_linear_modules

###### torch.jit.quantized.QuantizedLinear
###### torch.jit.QuantizedLinearFP16

###### torch.jit.quantized.QuantizedGRU
###### torch.jit.quantized.QuantizedGRUCell
###### torch.jit.quantized.QuantizedLSTM
###### torch.jit.quantized.QuantizedLSTMCell

In [20]:
url = f"{FAST_API_HOST}/similar_docs"

payload = json.dumps({
    "text": "Могу ли я использовать эти API в pytorch 2.3.0?\n# torch.jit.quantized APIs\ntorch.jit.quantized.quantize_rnn_cell_modules\ntorch.jit.quantized.quantize_rnn_modules\ntorch.jit.quantized.quantize_linear_modules\ntorch.jit.quantized.QuantizedLinear\ntorch.jit.QuantizedLinearFP16\ntorch.jit.quantized.QuantizedGRU\ntorch.jit.quantized.QuantizedGRUCell\ntorch.jit.quantized.QuantizedLSTM\ntorch.jit.quantized.QuantizedLSTMCell"
})
headers = {
    'Content-Type': 'application/json',
    'Cache-Control': 'no-cache',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip,deflate',
    'Connection': 'keep-alive'
}

response = requests.request("GET", url, headers=headers, data=payload)

for element in response.json():
    beautify_string = bytes(element['page_content'], "utf-8").decode("unicode_escape")
    print(beautify_string)
    print("\n ----------END_OF_DOC---------- \n")

pytorch
2.2.2
2.3.0
2024-04-26
Backwards Incompatible Changes

Remove deprecated torch.jit.quantized APIs (#118406)
All functions and classes under torch.jit.quantized will now raise an error if called/instantiated. This API has long been deprecated in favor of torch.ao.nn.quantized.

2.2	2.3
# torch.jit.quantized APIs

torch.jit.quantized.quantize_rnn_cell_modules

torch.jit.quantized.quantize_rnn_modules
torch.jit.quantized.quantize_linear_modules

torch.jit.quantized.QuantizedLinear
torch.jit.QuantizedLinearFP16

torch.jit.quantized.QuantizedGRU
torch.jit.quantized.QuantizedGRUCell
torch.jit.quantized.QuantizedLSTM
torch.jit.quantized.QuantizedLSTMCell
# Corresponding torch.ao.quantization APIs

torch.ao.nn.quantized.dynamic.RNNCell

torch.ao.quantization.quantize_dynamic APIs

torch.ao.nn.quantized.dynamic.Linear

torch.ao.nn.quantized.dynamic.GRU
torch.ao.nn.quantized.dynamic.GRUCell
torch.ao.nn.quantized.dynamic.LSTM

 ----------END_OF_DOC---------- 

pytorch
2.2.2
2.3.0
2024-04-

### 7. Отправляем запрос к YaGPT

In [29]:
url = f"{FAST_API_HOST}/llm?folder_id={FOLDER_ID}&token={TOKEN}"

payload = json.dumps({
    "text": "Могу ли я использовать эти API в pytorch 2.3.0?\n# torch.jit.quantized APIs\ntorch.jit.quantized.quantize_rnn_cell_modules\ntorch.jit.quantized.quantize_rnn_modules\ntorch.jit.quantized.quantize_linear_modules\ntorch.jit.quantized.QuantizedLinear\ntorch.jit.QuantizedLinearFP16\ntorch.jit.quantized.QuantizedGRU\ntorch.jit.quantized.QuantizedGRUCell\ntorch.jit.quantized.QuantizedLSTM\ntorch.jit.quantized.QuantizedLSTMCell"
})
headers = {
    'Content-Type': 'application/json',
    'Cache-Control': 'no-cache',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip,deflate',
    'Connection': 'keep-alive'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.json())

Чтобы определить, можно ли использовать указанные API в PyTorch 2.3.0, необходимо учесть следующие моменты:
1. PyTorch версии 2.3.0 вышла в июне 2020 года, а последняя версия PyTorch на момент публикации этого текста (21 октября 2023 года) — 1.12.1.*, поэтому API из PyTorch 2.3.0 могут быть недоступны в более новых версиях.
1. API для квантизации слоёв и моделей (*torch.jit.qantized.*) доступны в PyTorch начиная с версии 1.5.0.  

Исходя из этих фактов, можно предположить, что API, указанные в запросе, могут быть доступны в PyTorch версии  2.3.0. Однако для точного ответа на этот вопрос необходимо удостовериться в этом напрямую.

Самый простой способ проверить доступность API в PyTorch — воспользоваться *pip freeze* или *pip list*, чтобы вывести список установленных пакетов. Затем нужно найти PyTorch в списке и проверить его версию.
**Пример использования *pip freeze***:

```
pip freeze > pip_freeze.txt
```

Затем можно найти в списке PyTorch и посмотреть его версию, используя команду 

### 8. Отправляем запрос к YaGPT с использованием найденных раннее релевантных документов

In [32]:
url = f"{FAST_API_HOST}/llm_rag?folder_id={FOLDER_ID}&token={TOKEN}"

payload = json.dumps({
    "text": "Могу ли я использовать эти API в pytorch 2.3.0?\n# torch.jit.quantized APIs\ntorch.jit.quantized.quantize_rnn_cell_modules\ntorch.jit.quantized.quantize_rnn_modules\ntorch.jit.quantized.quantize_linear_modules\ntorch.jit.quantized.QuantizedLinear\ntorch.jit.QuantizedLinearFP16\ntorch.jit.quantized.QuantizedGRU\ntorch.jit.quantized.QuantizedGRUCell\ntorch.jit.quantized.QuantizedLSTM\ntorch.jit.quantized.QuantizedLSTMCell"
})
headers = {
    'Content-Type': 'application/json',
    'Cache-Control': 'no-cache',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip,deflate',
    'Connection': 'keep-alive'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.json()["output_text"])

На основе приведённого вами фрагмента можно сделать вывод, что часть перечисленных функций и классов в PyTorch 2.3.0 больше не поддерживается и при вызове или создании экземпляра будет вызывать ошибку.

Для уточнения деталей и получения более точной информации рекомендуется обратиться к документации PyTorch или обратиться к сообществу разработчиков PyTorch на официальном сайте https://discuss.pytorch.org/.

**Важно отметить, что это лишь фрагмент документации PyTorch от 26 апреля 2024 года, а не полная документация по всем версиям PyTorch.** Для получения полной информации о поддерживаемых версиях и изменениях рекомендуется обратиться к актуальной документации на официальном сайте PyTorch.
